# 使用 Dask 进行多 GPU 计算

利用 GPU 加速您的工作负载可以带来数量级的性能提升，但一旦您的工作负载完全利用了设备，您就会开始遇到新的性能瓶颈。

这就是多 GPU 和多节点工作负载发挥作用的地方。可以使用多个 GPU 一起工作，并看到另一个性能的飞跃。

在我们深入研究多 GPU 工作负载之前，我想提醒一下，分布式计算可能会增加代码的复杂性。本章讨论的工具尽其所能地减轻分布式计算的负担，但在我们开始扩展之前，我们应该确保已经在单个 GPU 上榨干了每一滴性能。

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

## Dask

[Dask](https://dask.org) 是一个用于扩展 Python 代码的库。在其核心，Dask 将您的 Python 代码转换为函数调用、输入和输出的计算图。然后它有一系列调度器可以用来并行执行这个图。这里我们将重点关注 Dask 的分布式调度器。

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
# 提交一个函数在 Dask 集群上执行
f = client.submit(lambda: 10 + 1)
f.result()

In [ ]:
# 使用高级集合 API 在集群上分布熟悉的工作
import dask.array as da
arr = da.random.random((1000, 1000), chunks=(100, 100))
arr.mean().compute()

Dask 并不太关心您的代码在做什么，它只是尝试用它的工作进程池尽可能快地运行图。因为我们所有的 GPU 计算都是在 Python 中完成的，所以 Dask 也可以分发我们的 GPU 代码。

In [ ]:
client.close()

### 分布式集群

为了让 Dask 将图分发到多台机器上，它需要一个调度器进程和多个工作进程。我们可以手动启动这些进程，可以通过 CLI 命令 `dask-scheduler` 和 `dask-worker`，也可以使用 Dask 的任何集群管理器。

#### 集群管理器

只要您有 Python 环境、网络连接并且可以启动调度器和工作进程，Dask 就可以在任何计算环境中运行。

为了使创建 Dask 集群的体验保持一致，有许多集群管理器类可以导入和实例化，它们会为您构建集群。

大多数人首先接触的是 `LocalCluster`。当您创建这个类的实例时，它会检查本地计算机上可用的 CPU 和内存资源，并自动为调度器和适当数量的工作进程创建子进程。

In [ ]:
from dask.distributed import LocalCluster

cluster = LocalCluster()
cluster

这对于尝试 Dask 和利用本地机器上所有可用的 CPU 核心来说非常好。

一旦您准备超越计算机的限制，就有针对 HPC 平台（如 SLURM、PBS 和 SGE）的集群管理器。还有针对 Kubernetes、Hadoop 和公共云提供商（包括 Amazon Web Services、Microsoft Azure 和 Google Cloud Platform）的集群管理器。

我们看到人们使用最多的集群管理器是 `SSHCluster` 类，它打开到网络上其他机器的安全 shell 连接，并在这些机器上启动 Dask 进程。这对于远程利用服务器甚至是未使用的其他台式机来说非常好。

```python
from dask.distributed import SSHCluster

cluster = SSHCluster([
    "localhost",  # 在此主机上启动调度器
    "10.0.0.2",   # 在此主机上启动第一个工作进程
    "10.0.0.3",   # 在此主机上启动第二个工作进程
    ...           # 等等
])
```

In [ ]:
cluster.close()

#### Dask CUDA

在使用 GPU 和 Dask 时，我们需要记住一些事情。每个 Dask 工作进程需要恰好有一个 GPU，所以如果您的机器有多个 GPU，您需要每个设备一个工作进程。为了让 Dask 工作进程能够成功利用 GPU，还需要做一些其他事情。为了简化用户的操作，您可以使用 `dask-cuda` Python 包中的工具。

Dask CUDA 包有一个名为 `LocalCUDACluster` 的集群管理器和一个名为 `dask-cuda-worker` 的替代工作进程 CLI 命令。这两者都会检查您的硬件，为每个 GPU 启动一个工作进程，并正确配置每个工作进程只使用其分配的设备。

In [ ]:
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
cluster

In [ ]:
client = Client(cluster)

也可以配置其他利用 HPC 和云功能的集群管理器使用 Dask CUDA 工作进程而不是常规工作进程。

一旦我们有了带 GPU 工作进程的 Dask 集群，我们就可以手动提交用 Numba 编写的 CUDA 内核在这些 GPU 上执行。

In [ ]:
from numba import cuda

@cuda.jit
def some_kernel():
    i = 0
    while i < 1_000_000:
        i += 1

f = client.submit(some_kernel[1024*1024, 1024])
f

### 高级集合

幸运的是，我们不必在 Dask 中手动完成所有事情。Dask 有高级集合的概念，它们实现了流行 Python 包的 API，但将数据结构和任务分块/分区，以便它们可以在 Dask 集群上运行。通常人们使用 `dask.array`（遵循 NumPy API）、`dask.dataframe`（遵循 Pandas API）和 `dask.ml`（遵循 Scikit-Learn API）。

这种方法可能听起来很熟悉，我们已经看到 RAPIDS 库模仿这些库的 API 来提供加速计算。Dask 做同样的事情，但是用于分布式计算。这种方法的一个好处是我们可以将它们结合起来，获得我们已经熟悉的工具的分布式和加速计算。

当 `dask.dataframe` 创建一个 DataFrame 时，它构建了一个由许多较小的 Pandas DataFrame 组成的任务图。然后像计算序列的平均值这样的操作首先会在每个 Pandas DataFrame 上执行，然后将结果聚合以获得总体平均值。但是 Dask 不限于在其 DataFrame 集合中使用 Pandas，它也可以利用其他遵循 Pandas API 的库，比如 cuDF。

cuDF 带有一个有用的辅助库，用于构建由 cuDF DataFrame 组成的 Dask DataFrame，我们可以像之前看到的那样加载数据并执行操作。

In [ ]:
import dask
import cudf
import dask_cudf

In [ ]:
@dask.delayed
def gen_partition():
    return cudf.datasets.timeseries()

gddf = dask_cudf.from_delayed([gen_partition() for i in range(30)])
gddf

In [ ]:
gddf.head()

In [ ]:
len(gddf)

In [ ]:
gddf.groupby("name").x.mean().compute()

但现在我们的 DataFrame 分布在所有 GPU 上，计算可以利用所有硬件的性能。

### 通信

在第1章探索 Numba CUDA 时，我们看到将数据从 CPU 内存移动到 GPU 内存时会有性能损失。在 GPU 内存之间和不同机器上的 GPU 之间移动数据时也是如此。

默认情况下，Dask 使用自定义 TCP 协议进行工作进程之间的通信。这意味着从一个 GPU 到另一个 GPU 的任何内存传输都必须通过 PCI-e 通道返回到 CPU，进入操作系统的网络栈以路由到其目的地。如果目标 GPU 在同一台机器上，它将沿着 PCI-e 通道返回到 GPU。如果它位于另一台机器上，它将通过 IP 网络（很可能通过以太网连接）传输。

在两个 GPU 在主板上相邻的情况下，这是非常浪费的。它们甚至可能通过 NVLINK 直接连接，或者至少连接到主板上的同一个 PCI-e 交换机。通过 CPU 路由每次传输是浪费的，这就是 UCX 的用武之地。

#### UCX

[UCX](https://openucx.org/) 是一个网络协议，它可以检查系统的拓扑结构并通过加速硬件找到最优路径。如果两个 GPU 通过 NVLINK 连接，那么 UCX 将使用它来传输数据，如果它们连接到同一个 PCI-e 交换机，那就是次优选择。如果 GPU 在两台不同的机器上，但这些机器有 Infiniband 网卡，那么 UCX 可以利用 Infiniband 上的 RDMA 在 GPU 之间直接传输数据。

UCX 会尽其所能在两个位置之间尽可能直接和高效地传输数据，最终才会退回到 TCP。

#### Dask 通信协议

Dask 支持用户可以配置的替代通信协议。这包括 UCX（我们可以利用它获得更好的性能），也包括其他协议，如 websockets，由于更容易代理，在现代系统架构中可能更灵活。

如果我们将 UCX 与我们的 GPU 工作进程一起使用，并且有加速网络硬件（如 NVLINK 或 Infiniband），那么我们可以看到 GPU 工作进程之间的内存传输时间大大减少。

### 资源注释

我想在 Dask 和 GPU 方面最后要讨论的主题是注释。Dask 有一个功能，可以为任务图中的每个任务添加工作进程需要具备的要求注释，以便能够运行它。

当我们启动工作进程时，我们也可以添加我们的资源标签，这样调度器就可以将适当的任务放在适当的工作进程上。当我们的工作进程配置各不相同时，这个功能最有用。


```console
$ dask-cuda-worker scheduler:8786 --resources "GPU=2"
```

在您的任务图中可能有一些步骤，在中间计算期间内存使用量会大幅增加。将这些任务引导到比其他工作进程有更多内存的工作进程可能会有帮助。

如果不是所有工作进程都有 GPU，我们也可以将此用于 GPU 工作。让大多数任务在常规 Dask 工作进程上运行，但也有几个 GPU 工作进程来运行已经优化为在 GPU 上运行的步骤，这是合理的。

如果您有一个现有的利用 Dask 的工作负载，并且想要尝试使用 GPU，这可能最有用。您可以添加一个有 GPU 的工作进程，选择工作流中的一些任务用 Numba 优化，并注释这些任务只在您的 GPU 工作进程上运行。

```python
foo = client.submit(some_non_gpu_function)

with dask.annotate(resources={'GPU': 1}):
    bar = client.submit(a_gpu_function, foo)
    
baz = client.submit(another_non_gpu_function, bar)
```